# 1.

In [11]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


import math
import numpy as np
import pandas as pd

#import scikitplot
import seaborn as sns
from matplotlib import pyplot

In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/FER 2013 default/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical'
        )

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/FER 2013 default/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical'
        )

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
# create model structure
emotion_model = Sequential()

# the model so far outputs 3D feature maps (height, width, features)
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# this converts our 3D feature maps to 1D feature vectors
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

2023-03-31 17:24:56.832828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/roma/anaconda3/envs/deep_learning/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [7]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=200,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/200


/var/folders/cb/nv527ffd4sn4f92077l2twqr0000gn/T/ipykernel_59379/957706480.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


448/448 [==============================] - 74s 165ms/step - loss: 0.3435 - accuracy: 0.8776 - val_loss: 1.2452 - val_accuracy: 0.6179
Epoch 2/200
448/448 [==============================] - 72s 161ms/step - loss: 0.3376 - accuracy: 0.8796 - val_loss: 1.2691 - val_accuracy: 0.6145
Epoch 3/200
448/448 [==============================] - 72s 161ms/step - loss: 0.3243 - accuracy: 0.8842 - val_loss: 1.2860 - val_accuracy: 0.6159
Epoch 4/200
448/448 [==============================] - 72s 160ms/step - loss: 0.3206 - accuracy: 0.8843 - val_loss: 1.2926 - val_accuracy: 0.6183
Epoch 5/200
448/448 [==============================] - 72s 161ms/step - loss: 0.3024 - accuracy: 0.8928 - val_loss: 1.2950 - val_accuracy: 0.6144
Epoch 6/200
448/448 [==============================] - 72s 160ms/step - loss: 0.3080 - accuracy: 0.8897 - val_loss: 1.3021 - val_accuracy: 0.6161
Epoch 7/200
448/448 [==============================] - 72s 160ms/step - loss: 0.2880 - accuracy: 0.8975 - val_loss: 1.3382 - val_accurac

In [8]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model_3_epochs200.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model_3_epochs200.h5')